Importacion de Librerias

In [1]:
import pandas as pd
import numpy as np
import ast
from dateutil import parser
import pyarrow as pa
import pyarrow.parquet as pq
from textblob import TextBlob

Funciones


In [ ]:
# La funcion ReadANDdicc lee el archivo json y retorna una lista diccionarios por cada fila del archivo leido.
def ReadANDdicc(Path):
    List = []
    with open(Path, 'r', encoding ="Latin-1") as File:
        for Line in File:
            try:
        # Usar ast.literal_eval() para convertir cada línea en un diccionario que sera agregado a la lista
                raw_data_dicc = ast.literal_eval(Line)
                List.append(raw_data_dicc)
            except ValueError as e:
                # Aviso en caso de que el codigo encuentre un error en la linea
                print(f"Error en la línea: {Line}")
                continue
    return List

In [ ]:
# date_extractor extrae la fecha dentro de un registro tipo string eliminando Posted 
def date_extractor(raw_date):
    if isinstance(raw_date, str):
        return parser.parse(raw_date.replace("Posted ", ""), fuzzy=True)
    else:
        return raw_date
    

In [ ]:
#Genera un analisis de sentimientos por cada comentario (review)
def sentiment_analyzer(review):
  analysis_sentiment = TextBlob(str(review))
  # Asigna un valor según el análisis de sentimiento
  if analysis_sentiment.polarity < -0.2:
    return 0  # Sentimiento negativo (Malo)
  elif -0.2 <= analysis_sentiment.polarity <= 0.2:
    return 1  # Sentimiento neutral
  else:
    return 2  # Sentimiento positivo

Procesamiento de datos


In [9]:
# Definicion de las rutas de cada archivo Json  a usar
Review_path = "Data/australian_user_reviews.json"
Items_path = "Data/australian_users_items.json"
Games_path ="Data/output_steam_games.json"

Reviews


In [ ]:
# Iniciamos el proceso de extraccion de los datos proporcionados en el archivo json

#Con la siguiente funcion Extraemos la lista diccionarios que conforman el archivo
Review_diccs = ReadANDdicc(Review_path)

# Creamos un Dataframe donde podamos manipular los datos
df_users_reviews = pd.DataFrame(Review_diccs)

# Visualizamos la estructura del conjunto de datos y podremos notar datos anidados en la columna reviews
df_users_reviews

In [ ]:
# Con ayuda del metodo explode deanidaremos estos datos y los transformamos en 
# nuevas columnas que esten a la par de las anteriores

df_users_reviews = df_users_reviews.explode("reviews").reset_index()
df_users_reviews = df_users_reviews.drop(columns="index")

# Unimos estas nuevas columnas a nuestro dataframe base para mantener todo en un solo dataframe 
df_users_reviews = pd.concat([df_users_reviews, pd.json_normalize(df_users_reviews['reviews'])], axis=1)
df_users_reviews.head()

,user_id,user_url,reviews,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20...",,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011....",,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011...",,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014....",,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2...",,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...


In [ ]:
# Usamos la funcion sentiment_analyzer para crear un analisis de sentimiento tomando de referencia
# nuestra nueva columna review y le creamos su propia columna donde organizaremos la data .
df_users_reviews['sentiment_analysis'] = df_users_reviews['review'].apply(sentiment_analyzer)


In [ ]:
df_users_reviews

,user_id,posted,item_id,helpful,recommend,review,sentiment_analysis,year
0,76561197970982479,2011-11-05,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,1,2011
1,76561197970982479,2011-07-15,22200,No ratings yet,True,It's unique and worth a playthrough.,2,2011
2,76561197970982479,2011-04-21,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,1,2011
3,js41637,2014-06-24,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,1,2014
4,js41637,2013-09-08,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,1,2013
...,...,...,...,...,...,...,...,...
59328,76561198312638244,2024-07-10,70,No ratings yet,True,a must have classic from steam definitely wort...,2,NaN
59329,76561198312638244,2024-07-08,362890,No ratings yet,True,this game is a perfect remake of the original ...,1,NaN
59330,LydiaMorley,2024-07-03,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,1,NaN
59331,LydiaMorley,2024-07-20,730,No ratings yet,True,:D,2,NaN


In [ ]:
#Agregamos la columna año tomando en cuenta la fecha en la columna posted y elminamos las columnas no necesarias
df_users_reviews['year'] = df_users_reviews['posted'].str.extract(r'(\d{4})')
df_users_reviews = df_users_reviews.drop(columns=(["user_url","reviews","funny","last_edited"]))

In [ ]:
# Con la funcion date_analyzer modificamos todos los elementos de la columna posted y los cambiamos a un formato de fecha
# eliminando la palabra "Posted" de cada registro 
df_users_reviews['posted'] = df_users_reviews['posted'].apply(date_extractor)


In [ ]:
#Contamos el numero de nulos en las columnas
null_data_reviews = df_users_reviews.isna().sum()
null_data_reviews

user_id                   0
posted                   28
item_id                  28
helpful                  28
recommend                28
review                   28
sentiment_analysis        0
year                  10147
dtype: int64

In [ ]:
#Eliminamos los nulos para quedarnos con datos concisos
df_users_reviews = df_users_reviews.dropna()

In [ ]:
#Hacemos una conversion del dataframe a un archivo csv
reviews_csv = df_users_reviews.to_csv('data/reviews_csv.csv',index=False)

In [ ]:
df_users_reviews

In [ ]:
# Realizamos el proceso de transformacion del archivo csv a parquet
reviews_csv = pd.read_csv('data/reviews_csv.csv')
PyArrow_table_reviews = pa.Table.from_pandas(reviews_csv)
pq.write_table(PyArrow_table_reviews,'data/reviews_file.parquet')

In [ ]:
# Leer el archivo Parquet en una tabla de PyArrow
PyArrow_table_reviews = pq.read_table('Data/reviews_file.parquet')

# Convertir la tabla de PyArrow a un DataFrame de pandas
reviews_parquet = PyArrow_table_reviews.to_pandas()
reviews_parquet

Items

In [ ]:
# Extraemos la lista diccionarios que conforman el archivo
Items_diccs = ReadANDdicc(Items_path)

# Creamos un Dataframe donde podamos manipular los datos
df_users_Items = pd.DataFrame(Items_diccs)

# Visualizamos la estructura del conjunto de datos y podremos notar datos anidados en la columna items
df_users_Items

In [ ]:
# Con ayuda del metodo explode deanidaremos estos datos y los transformamos en 
# nuevas columnas que esten a la par de las anteriores

df_users_Items = df_users_Items.explode("items").reset_index()
df_users_Items = df_users_Items.drop(columns="index")

# Unimos estas nuevas columnas a nuestro dataframe base para mantener todo en un solo dataframe 
df_users_Items = pd.concat([df_users_Items, pd.json_normalize(df_users_Items['items'])], axis=1)
df_users_Items.head()

In [ ]:
# Eliminamos columnas que no van a ser necesarias
df_users_Items = df_users_Items.drop(columns=(["items","user_url"]))

In [ ]:
# Contamos los registros nulos 
df_users_Items.isnull().sum()

In [ ]:
# Eliminamos los registros nulos 
df_users_Items = df_users_Items.dropna()

In [ ]:
# Transformamos los registros con "" por registros vacios
df_users_Items.loc[:, 'item_id'] = df_users_Items['item_id'].str.replace('"', '')
df_users_Items.loc[:, 'items_id'] = df_users_Items['item_id'].astype(int)

In [ ]:
# Convertimos el Dataframe a un archivo c
items_csv= df_users_Items.to_csv('Data/items_csv.csv',index=False) #convertimos a csv


In [ ]:
# Se Lee el archivo CSV usando pandas
items_csv = pd.read_csv('Data/items_csv.csv')

# Convertimos el DataFrame de pandas a una tabla de PyArrow
PyArrow_table_items = pa.Table.from_pandas(items_csv)

# Escribir la tabla en un archivo Parquet
pq.write_table(PyArrow_table_items, 'items_file.parquet')

Games

In [2]:
# Lee el archivo output_steam_games y carga los datos en un DataFrame
df_steam_games = pd.read_json("Data/output_steam_games.json", lines=True)

# Elimina registros con valores NaN en todos los campos
df_steam_games.dropna(how='all', inplace=True)

# Muestra el DataFrame con los valores eliminados
df_steam_games 

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL
88312,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,0.0,670290.0,Poolians.com
88313,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,0.0,767400.0,彼岸领域
88314,None,None,Log Challenge,None,http://store.steampowered.com/app/773570/Log_C...,None,"[Action, Indie, Casual, Sports]",http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,0.0,773570.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


In [3]:
# Eliminamos columnas que no van a ser necesarias
df_steam_games = df_steam_games.drop(columns=(["url","reviews_url","publisher","early_access","specs"]))

In [4]:
# Se agrega una nueva columna que nos indique el año de lanzamiento
df_steam_games['release_year'] = df_steam_games['release_date'].str.extract(r'(\d{4})')

# Visualizamos el dataframe para observar nuevas trasnformaciones 
df_steam_games 

,genres,app_name,title,release_date,tags,price,id,developer,release_year
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140.0,Kotoshiro,2018
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",Free To Play,643980.0,Secret Level SRL,2018
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",Free to Play,670290.0,Poolians.com,2017
88313,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400.0,彼岸领域,2017
88314,None,Log Challenge,None,None,"[Action, Indie, Casual, Sports]",2.99,773570.0,None,NaN
...,...,...,...,...,...,...,...,...,...
120440,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.99,773640.0,"Nikita ""Ghost_RUS""",2018
120441,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",4.99,733530.0,Sacada,2018
120442,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",1.99,610660.0,Laush Dmitriy Sergeevich,2018
120443,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",4.99,658870.0,"xropi,stev3ns",2017


In [5]:
# Contamos los elementos faltantes
nan_data_games = df_steam_games.isna().sum()
nan_data_games

genres          3283
app_name           2
title           2050
release_date    2067
tags             163
price           1377
id                 2
developer       3299
release_year    2168
dtype: int64

In [6]:
# Renombramos la columna id por item_id 
df_steam_games = df_steam_games.rename(columns={'id': 'item_id'})

In [7]:
# Contamos los registros de la columna price que son str
str_prices = df_steam_games[df_steam_games['price'].apply(lambda x: isinstance(x, str))]
str_prices['price'].value_counts()


price
Free                             905
Free to Play                     520
Free To Play                     462
Free Mod                           4
Free Demo                          3
Play Now                           2
Third-party                        2
Play for Free!                     2
Play WARMACHINE: Tactics Demo      1
Install Theme                      1
Install Now                        1
Free HITMAN™ Holiday Pack          1
Play the Demo                      1
Starting at $499.00                1
Starting at $449.00                1
Free to Try                        1
Free Movie                         1
Free to Use                        1
Name: count, dtype: int64

In [8]:
# Transformamos los elementos anteriores y los reemplazamos por 0
df_steam_games.loc[df_steam_games['price'].apply(lambda x: isinstance(x, str)), 'price'] = 0

In [9]:
# Transformamos en 0  a los registros que no poseen Id
df_steam_games['item_id'] = df_steam_games['item_id'].fillna(0).astype(int)

# v en enteros los ID del DataFrame df_steam_games_original
df_steam_games['item_id'] = df_steam_games['item_id'].astype(int)

# Observamos el Dataframe después de las transformaciones
df_steam_games

,genres,app_name,title,release_date,tags,price,item_id,developer,release_year
88310,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140,Kotoshiro,2018
88311,"[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",0,643980,Secret Level SRL,2018
88312,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",0,670290,Poolians.com,2017
88313,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400,彼岸领域,2017
88314,None,Log Challenge,None,None,"[Action, Indie, Casual, Sports]",2.99,773570,None,NaN
...,...,...,...,...,...,...,...,...,...
120440,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.99,773640,"Nikita ""Ghost_RUS""",2018
120441,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",4.99,733530,Sacada,2018
120442,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",1.99,610660,Laush Dmitriy Sergeevich,2018
120443,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",4.99,658870,"xropi,stev3ns",2017


In [11]:
#Convertir la columna 'Genres' a cadenas (si no lo es)
df_steam_games['genres'] = df_steam_games['genres'].astype(str)

#Crear una nueva columna 'Genres' con solo el primer valor de la lista
df_steam_games['genres'] = df_steam_games['genres'].str.strip('[]').str.split(',').str[0].str.replace("'", '')

#Convertir la columna 'Genres' a tipo object
df_steam_games['genres'] = df_steam_games['genres'].astype(object)

In [19]:
#Eliminamos las columnas innecesarias 
df_steam_games

,genres,app_name,title,release_date,tags,price,item_id,developer,release_year
88310,Action,Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.99,761140,Kotoshiro,2018
88311,Free to Play,Ironbound,Ironbound,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",0,643980,Secret Level SRL,2018
88312,Casual,Real Pool 3D - Poolians,Real Pool 3D - Poolians,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",0,670290,Poolians.com,2017
88313,Action,弹炸人2222,弹炸人2222,2017-12-07,"[Action, Adventure, Casual]",0.99,767400,彼岸领域,2017
88314,None,Log Challenge,None,None,"[Action, Indie, Casual, Sports]",2.99,773570,None,NaN
...,...,...,...,...,...,...,...,...,...
120440,Casual,Colony On Mars,Colony On Mars,2018-01-04,"[Strategy, Indie, Casual, Simulation]",1.99,773640,"Nikita ""Ghost_RUS""",2018
120441,Casual,LOGistICAL: South Africa,LOGistICAL: South Africa,2018-01-04,"[Strategy, Indie, Casual]",4.99,733530,Sacada,2018
120442,Indie,Russian Roads,Russian Roads,2018-01-04,"[Indie, Simulation, Racing]",1.99,610660,Laush Dmitriy Sergeevich,2018
120443,Casual,EXIT 2 - Directions,EXIT 2 - Directions,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",4.99,658870,"xropi,stev3ns",2017


In [20]:
games_csv = df_steam_games.to_csv('Data/games_csv.csv',index=False)

In [21]:
# Leer el archivo CSV en un DataFrame de pandas
games_csv = pd.read_csv('Data/games_csv.csv')

# Convertir el DataFrame de pandas a una tabla de PyArrow
PyArrow_table_items = pa.Table.from_pandas(games_csv)

# Escribir la tabla en un archivo Parquet
pq.write_table(PyArrow_table_items, 'Data/games_file.parquet')